# Tutorial 2 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Install TaskGen

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Takes in menu items and curates them based on profitability'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with fictitious pricing for each of them')
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')
```

In [3]:
def random_number_from_string(input_string):
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [4]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in <list_of_dish_names> and outputs price '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output
    
fp = Function(fn_description = "Takes in <list_of_dish_names: list> and outputs price", 
              output_format = {"Price": "Dish name as key and price as value"}, 
              examples = {"list_of_dish_names": ["Pasta", "Pizza"], "Price": [{'Pasta': '$5'}, {'Pizza':'$2'}]},
              external_fn = dish_price)

In [5]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

In [6]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Takes in menu items and curates them based on profitability'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with fictitious pricing for each of them').assign_functions([fp])
    ]

In [7]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [8]:
output = my_agent.run('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')

Subtask identified: Generate interesting dish names and descriptions for Italian cuisine
Calling function Creative Writer with parameters {'instruction': 'Generate interesting dish names and descriptions for Italian cuisine'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Research popular Italian dishes
Getting LLM to perform the following task: Research popular Italian dishes
> I have researched popular Italian dishes such as spaghetti carbonara, lasagna, risotto alla milanese, pizza margherita, and tiramisu.

Subtask identified: Brainstorm creative dish names
Getting LLM to perform the following task: Brainstorm creative dish names
> 1. Bella Vita Pasta - A delightful combination of al dente pasta, sun-dried tomatoes, olives, and fresh basil in a light garlic sauce. 2. Vesuvio Veal - Tender veal medallions sautéed with mushrooms, artichokes, and capers in a white wine sauce. 3. Amore Arancini - Crispy risotto balls stuffed with mozzarella cheese and served with a 

In [9]:
output = my_agent.reply_user()

{'Menu Items': [{'Name': 'Bella Vita Pasta', 'Description': 'A delightful combination of al dente pasta, sun-dried tomatoes, olives, and fresh basil in a light garlic sauce.', 'Ingredients': 'spaghetti, sun-dried tomatoes, olives, fresh basil, garlic', 'Price': '$10'}, {'Name': 'Vesuvio Veal', 'Description': 'Tender veal medallions sautéed with mushrooms, artichokes, and capers in a white wine sauce.', 'Ingredients': 'veal, mushrooms, artichokes, capers, white wine', 'Price': '$5'}, {'Name': 'Amore Arancini', 'Description': 'Crispy risotto balls stuffed with mozzarella cheese and served with a zesty marinara dipping sauce.', 'Ingredients': 'risotto, mozzarella cheese, marinara sauce', 'Price': '$5'}, {'Name': 'Capri Calamari', 'Description': 'Lightly fried calamari rings tossed with cherry peppers, garlic, and parsley, served with a side of lemon aioli.', 'Ingredients': 'calamari, cherry peppers, garlic, parsley, lemon aioli', 'Price': '$4'}, {'Name': 'Dolce Vita Tiramisu', 'Descriptio

In [10]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Research popular Italian dishes
I have researched popular Italian dishes such as spaghetti carbonara, lasagna, risotto alla milanese, pizza margherita, and tiramisu.

Subtask: Brainstorm creative dish names
1. Bella Vita Pasta - A delightful combination of al dente pasta, sun-dried tomatoes, olives, and fresh basil in a light garlic sauce. 2. Vesuvio Veal - Tender veal medallions sautéed with mushrooms, artichokes, and capers in a white wine sauce. 3. Amore Arancini - Crispy risotto balls stuffed with mozzarella cheese and served with a zesty marinara dipping sauce. 4. Capri Calamari - Lightly fried calamari rings tossed with che

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [11]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [12]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')

In [13]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Research popular Italian dishes
Getting LLM to perform the following task: Research popular Italian dishes
> 1. Spaghetti Carbonara: A classic Italian pasta dish made with spaghetti, eggs, pancetta, and cheese. 2. Margherita Pizza: A traditional pizza topped with tomato sauce, fresh mozzarella, basil, and olive oil. 3. Risotto ai Funghi: Creamy risotto cooked with mushrooms, onions, white wine, and Parmesan cheese. 4. Tiramisu: A popular Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone cheese, dusted with cocoa powder. 5. Osso Buco: Braised veal shanks cooked with vegetables, white wine, and broth, served with gremolata.

Subtask identified: Create unique names for Italian dishes
Getting LLM to perform the following task: Create unique names for Italian dishes
> 1. Bella Vita Pas

In [14]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Research popular Italian dishes
1. Spaghetti Carbonara: A classic Italian pasta dish made with spaghetti, eggs, pancetta, and cheese. 2. Margherita Pizza: A traditional pizza topped with tomato sauce, fresh mozzarella, basil, and olive oil. 3. Risotto ai Funghi: Creamy risotto cooked with mushrooms, onions, white wine, and Parmesan cheese. 4. Tiramisu: A popular Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone cheese, dusted with cocoa powder. 5. Osso Buco: Braised veal shanks cooked with vegetables, white wine, and broth, served with gremolata.

Subtask: Create unique names for Italian dishes
1. Bella

In [15]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Subtask identified: Research ingredients for Spaghetti Carbonara
Getting LLM to perform the following task: Research ingredients for Spaghetti Carbonara
> Spaghetti Carbonara ingredients: spaghetti, eggs, pancetta, cheese

Subtask identified: Research ingredients for Margherita Pizza
Getting LLM to perform the following task: Research ingredients for Margherita Pizza
> Margherita Pizza ingredients: pizza dough, tomato sauce, fresh mozzarella, basil, olive oil

Subtask identified: Research ingredients for Risotto ai Funghi
Getting LLM to perform the following task: Research ingredients for Risotto ai Funghi
> Risotto ai Funghi ingredients: Arborio rice, mushrooms, onions, white wine, Parmesan cheese

Subtask identified: Research ingredients for Tiramisu
Getting LLM to perform the following task: Research ingredients for Tiramisu
> Tiramisu ingredient

In [16]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Subtask identified: Generate prices for Bella Vita Pasta
Calling function dish_price with parameters {'list_of_dish_names': ['Bella Vita Pasta']}
> {'Price': {'Bella Vita Pasta': '$10'}}

Subtask identified: Generate prices for Vesuvio Delight
Calling function dish_price with parameters {'list_of_dish_names': ['Vesuvio Delight']}
> {'Price': {'Vesuvio Delight': '$10'}}

Subtask identified: Generate prices for Amore Risotto
Calling function dish_price with parameters {'list_of_dish_names': ['Amore Risotto']}
> {'Price': {'Amore Risotto': '$3'}}

Subtask identified: Generate prices for Dolce Vita Tiramisu
Calling function dish_price with parameters {'list_of_dish_names': ['Dolce Vita Tiramisu']}
> {'Price': {'Dolce Vita Tiramisu': '$9'}}

Subtask identified: Generate prices for Mare e Monti Osso Buco
Calling function dish_price with parameters {'

In [17]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay and explain why')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay and explain why'}

### Start of Inner Agent: Boss ###
Subtask identified: Select 3 profitable Italian dishes
Getting LLM to perform the following task: Select 3 profitable Italian dishes
> {'1': {'Name': 'Bella Vita Pasta', 'Description': 'A delightful dish featuring al dente pasta tossed in a creamy sun-dried tomato sauce, topped with grilled chicken and fresh basil.', 'Ingredients': 'Spaghetti, eggs, pancetta, cheese', 'Price': '$10'}, '2': {'Name': 'Dolce Vita Tiramisu', 'Description': 'An indulgent twist on the classic dessert, layers of espresso-soaked sponge cake and amaretto mascarpone, garnished with chocolate curls.', 'Ingredients': 'Ladyfingers, coffee, mascarpone cheese, cocoa powder, eggs, sugar', 'Price': '$9'}, '3': {'Name': 'Mare e Monti Osso Buco', 'Description': 'A surf and turf delight featuring tender veal shanks braised in a seafood-infused bro

In [18]:
# Get a targeted response from your agent, and add this to subtask_completed
output = my_agent.reply_user('Choose the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

Bella Vita Pasta, A delightful dish featuring al dente pasta tossed in a creamy sun-dried tomato sauce, topped with grilled chicken and fresh basil, Spaghetti, eggs, pancetta, cheese, $10
Dolce Vita Tiramisu, An indulgent twist on the classic dessert, layers of espresso-soaked sponge cake and amaretto mascarpone, garnished with chocolate curls, Ladyfingers, coffee, mascarpone cheese, cocoa powder, eggs, sugar, $9
Mare e Monti Osso Buco, A surf and turf delight featuring tender veal shanks braised in a seafood-infused broth, served with a zesty gremolata, Veal shanks, vegetables, white wine, broth, gremolata, $7


In [19]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Research popular Italian dishes
1. Spaghetti Carbonara: A classic Italian pasta dish made with spaghetti, eggs, pancetta, and cheese. 2. Margherita Pizza: A traditional pizza topped with tomato sauce, fresh mozzarella, basil, and olive oil. 3. Risotto ai Funghi: Creamy risotto cooked with mushrooms, onions, white wine, and Parmesan cheese. 4. Tiramisu: A popular Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone cheese, dusted with cocoa powder. 5. Osso Buco: Braised veal shanks cooked with vegetables, white wine, and broth, served with gremolata.

Subtask: Create unique names for Italian dishes
1. Bella

In [20]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

Bella Vita Pasta, A delightful dish featuring al dente pasta tossed in a creamy sun-dried tomato sauce, topped with grilled chicken and fresh basil, Spaghetti, eggs, pancetta, cheese, $10
Dolce Vita Tiramisu, An indulgent twist on the classic dessert, layers of espresso-soaked sponge cake and amaretto mascarpone, garnished with chocolate curls, Ladyfingers, coffee, mascarpone cheese, cocoa powder, eggs, sugar, $9
Mare e Monti Osso Buco, A surf and turf delight featuring tender veal shanks braised in a seafood-infused broth, served with a zesty gremolata, Veal shanks, vegetables, white wine, broth, gremolata, $7


In [21]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format
Bella Vita Pasta, A delightful dish featuring al dente pasta tossed in a creamy sun-dried tomato sauce, topped with grilled chicken and fresh basil, Spaghetti, eggs, pancetta, cheese, $10
Dolce Vita Tiramisu, An indulgent twist on the classic dessert, layers of espresso-soaked sponge cake and amaretto mascarpone, garnished with chocolate curls, Ladyfingers, coffee, mascarpone cheese, cocoa powder, eggs, sugar, $9
Mare e Monti Osso Buco, A surf and turf delight featuring tender veal shanks braised in a seafood-infused broth, served with a zesty gremolata, Ve

In [22]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ('Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format', 'Bella Vita Pasta, A delightful dish featuring al dente pasta tossed in a creamy sun-dried tomato sauce, topped with grilled chicken and fresh basil, Spaghetti, eggs, pancetta, cheese, $10\nDolce Vita Tiramisu, An indulgent twist on the classic dessert, layers of espresso-soaked sponge cake and amaretto mascarpone, garnished with chocolate curls, Ladyfingers, coffee, mascarpone cheese, cocoa powder, eggs, sugar, $9\nMare e Monti Osso Buco, A surf and turf delight featuring tender veal shanks braised in a seafood-infused broth, served with a zesty gremolata, Veal shanks, vegetables, white wine, broth, gremolata, $7')


In [23]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed: None
Is Task Completed: False
